In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)


In [2]:
df = pd.read_csv("var/shosetsu_dataset.csv")
df

,screenname,tweet,key
0,@amadeusu31224,江國香織「流しの下の骨」\n吉本ばなな「デッドエンドの思い出」「アムリタ」\n桐野夏生「グロ...,2019-05-29_21:00:00
1,@mana_wf,#名刺代わりの小説10選\n\nジェフリー・ディーヴァー『青い虚空』\nジャック・ロンドン『...,2019-05-29_21:00:00
2,@Kachi_ps,夜市/恒川光太郎\nリリエンタールの末裔/上田早夕里\n本と鍵の季節/米澤穂信\nななつのこ...,2019-05-29_21:00:00
3,@mana_wf,#名刺代わりの小説10選\n\n奈須きのこ『空の境界』\n淡路帆希『花守の竜の叙事詩』『ひと...,2019-05-29_21:00:00
4,Replying to\n @mana_wf,#名刺代わりの小説10選\n\n零崎軋識の人間ノック\n零崎曲識の人間人間\n零崎双識の人間...,2019-05-29_21:00:00
...,...,...,...
8785,@yuikapurupuru,#名刺代わりの小説10選\n\n悪魔の手毬唄/横溝正史\n犬神家の一族\nアナザヘヴン/飯田...,2020-10-12_09:00:00
8786,@odango345odango,#名刺代わりの小説10選\n池上永一/テンペスト\n上橋菜穂子/鹿の王\n小野不由美/東亰異...,2020-03-19_15:00:00
8787,@Jane_doe_silent,#名刺代わりの小説10選 #読書垢 \n虐殺器官/伊藤計劃\nジョンの世界/フィリップKディ...,2020-03-19_15:00:00
8788,@hyflower2,#名刺代わりの小説10選\n\n魔術師\n リンカーンシリーズ/ ジェフリーディーヴァー\n...,2020-03-19_15:00:00


In [3]:
import re
import mojimoji


tmp = []
exps = []
for tweet in df["tweet"]:
    if not isinstance(tweet, str):
        continue
        
    tweet = mojimoji.zen_to_han(tweet, kana=False)
    for line in tweet.split("\n"):
        line = line.strip()
        if "#" in line:
            continue
        if line == "":
            continue

        if "「" in line and "」" in line:
            a = re.search("「(.*?)」", line).group(1)
            b = re.sub("「.*?」", "", line)
            # print(a, b)
            tmp.append(a)
            tmp.append(b)
        elif "『" in line and "』" in line:
            a = re.search("『(.*?)』", line).group(1)
            b = re.sub("『.*?』", "", line)
            # print(a, b)
            tmp.append(a)
            tmp.append(b)
        elif "(" in line and ")" in line:
            a = re.search("\((.*?)\)", line).group(1)
            b = re.sub("\(.*?\)", "", line)
            # print(a, b)
            tmp.append(a)
            tmp.append(b)
        elif "/" in line and line.count("/") == 1:
            a, b = line.split("/")
            # print(a, b)
            tmp.append(a)
            tmp.append(b)
        elif "|" in line and line.count("|") == 1:
            a, b = line.split("|")
            #print(a, b)
            tmp.append(a)
            tmp.append(b)
        elif re.split("\s{1,}", line).__len__() == 2:
            a, b = re.split("\s{1,}", line)
            # print(a, b)
        else:
            # print("except", line)
            exps.append(line)
            
ok_size = len(tmp)/2
ng_size = len(exps)
print(ok_size/(ok_size+ng_size))

0.6361372831961067


In [4]:
noise = ["同上", "〃"]

In [5]:
names = pd.read_csv("var/auths.csv")["name"].tolist()
# names

In [6]:
titles = [x for x in np.array(tmp).flatten() if x not in names and x != "" and x not in noise]
titles = [x for x in np.array(tmp).flatten() if x not in names and x != "" and x not in noise]
# titles

In [7]:
from collections import Counter

In [8]:
#pd.set_option('display.min_rows', 200)
pd.set_option('display.max_rows', 490)
d = Counter(exps + titles)
a = pd.DataFrame({"name": list(d.keys()), "vals": list(d.values())})
a.sort_values(by=["vals"], ascending=False).reset_index()

,index,name,vals
0,473,十二国記,345
1,1048,こころ,290
2,16,銀河鉄道の夜,277
3,180,図書館戦争,270
4,541,人間失格,268
...,...,...,...
33136,5046,中上健次 - 枯木灘,1
33137,5047,・順列都市,1
33138,16171,ノーサイド・ゲーム,1
33139,16170,スウィフト,1


In [9]:
titles = a[a.vals >= 4].name.tolist()

In [10]:
auths = [x for x in np.array(tmp).flatten() if x in names and x != "" and x not in noise]
# auths

In [11]:
pd.set_option('display.min_rows', 200)
pd.set_option('display.max_rows', 200)
d = Counter(auths)
a = pd.DataFrame({"name": list(d.keys()), "vals": list(d.values())})
a.sort_values(by=["vals"], ascending=False).reset_index()

,index,name,vals
0,25,伊坂幸太郎,744
1,27,東野圭吾,686
2,26,有川浩,665
3,6,恩田陸,641
4,42,辻村深月,548
5,14,村上春樹,545
6,37,太宰治,514
7,72,森見登美彦,494
8,36,夏目漱石,462
9,41,小野不由美,425


In [12]:
total = []
for tweet in df["tweet"]:
    if not isinstance(tweet, str):
        continue
        
    block = []
    tweet = mojimoji.zen_to_han(tweet, kana=False)
    for line in tweet.split("\n"):
        line = line.strip()
        if "#" in line:
            continue
        if line == "":
            continue

        
        if "「" in line and "」" in line:
            a = re.search("「(.*?)」", line).group(1)
            b = re.sub("「.*?」", "", line)
            block.append(a)
            block.append(b)
        elif "『" in line and "』" in line:
            a = re.search("『(.*?)』", line).group(1)
            b = re.sub("『.*?』", "", line)
            block.append(a)
            block.append(b)
        elif "(" in line and ")" in line:
            a = re.search("\((.*?)\)", line).group(1)
            b = re.sub("\(.*?\)", "", line)
            block.append(a)
            block.append(b)
        elif "/" in line and line.count("/") == 1:
            a, b = line.split("/")
            block.append(a)
            block.append(b)
        elif "／" in line and line.count("／") == 1:
            a, b = line.split("／")
            block.append(a)
            block.append(b)
        elif "｜" in line and line.count("｜") == 1:
            a, b = line.split("｜")
            block.append(a)
            block.append(b)
        elif re.split("\s{1,}", line).__len__() == 2:
            a, b = re.split("\s{1,}", line)
            block.append(a)
            block.append(b)
        else:
            pass
    total.append(block)

In [13]:

total = [[x for x in block if x in names] for block in total ] 
total = [x for x in total if x.__len__() >= 1]
# total 

In [14]:

auth_cos = {}

for block in total:
    for auth in block:
        auth_cos[auth] = auth_cos.get(auth, []) + block

tmps = []
for auth, cos in list(auth_cos.items()):
    cos = Counter(cos)
    max_ = max(cos.values())
    tmp = pd.DataFrame({"co": list(cos.keys()), "val": list(cos.values())})
    tmp["auth"] = auth
    tmp = tmp[tmp.val >= 2]
    tmp["val"] /= max_
    tmps.append(tmp.sort_values(by=["val"], ascending=False)[:100])
ret = pd.concat(tmps)
ret = ret[["auth", "co", "val"]]
ret 

,auth,co,val
0,江國香織,江國香織,1.000000
6,江國香織,恩田陸,0.188406
19,江國香織,伊坂幸太郎,0.173913
32,江國香織,村上春樹,0.165217
4,江國香織,小川洋子,0.139130
41,江國香織,太宰治,0.124638
1,江國香織,吉本ばなな,0.124638
59,江國香織,森見登美彦,0.098551
49,江國香織,東野圭吾,0.098551
16,江國香織,山田詠美,0.095652


In [15]:
ret = ret[ret.apply(lambda x: x["auth"] != x["co"], axis=1)]
ret

,auth,co,val
6,江國香織,恩田陸,0.188406
19,江國香織,伊坂幸太郎,0.173913
32,江國香織,村上春樹,0.165217
4,江國香織,小川洋子,0.139130
41,江國香織,太宰治,0.124638
1,江國香織,吉本ばなな,0.124638
59,江國香織,森見登美彦,0.098551
49,江國香織,東野圭吾,0.098551
16,江國香織,山田詠美,0.095652
42,江國香織,夏目漱石,0.089855


In [16]:
ret.to_csv("tmp/recommends_auths.csv", index=None)

In [17]:
w = ret.groupby(by=["co"]).agg(val_sum=("val", "sum")).reset_index()
a = pd.merge(ret, w, on=["co"], how="left")

a["val_norm"] = a["val"]/np.log(a["val_sum"]+np.e)

tmps = []
for title, s in a.groupby(by=["auth"]):
    tmps.append(s.sort_values(by=["val_norm"], ascending=False)[:10])
    
a = pd.concat(tmps)
a.to_csv("tmp/recommends_auths_norm.csv", index=None)

In [18]:
a[a.auth == "村上春樹"]

,auth,co,val,val_sum,val_norm
1108,村上春樹,太宰治,0.178654,47.342068,0.045654
1109,村上春樹,夏目漱石,0.153132,38.102306,0.041285
1110,村上春樹,伊坂幸太郎,0.150812,59.302975,0.036539
1111,村上春樹,東野圭吾,0.124130,58.050495,0.030223
1112,村上春樹,恩田陸,0.119490,49.725607,0.030176
1114,村上春樹,三島由紀夫,0.088167,21.924883,0.027514
1121,村上春樹,ドストエフスキー,0.063805,9.710586,0.025319
1115,村上春樹,村上龍,0.082367,24.511053,0.024927
1113,村上春樹,森見登美彦,0.090487,36.031056,0.024743
1124,村上春樹,吉本ばなな,0.059165,8.422378,0.024544


In [18]:
total = []
for tweet in df["tweet"]:
    if not isinstance(tweet, str):
        continue
        
    block = []
    tweet = mojimoji.zen_to_han(tweet, kana=False)
    for line in tweet.split("\n"):
        line = line.strip()
        if "#" in line:
            continue
        if line == "":
            continue
        
        if "「" in line and "」" in line:
            a = re.search("「(.*?)」", line).group(1)
            b = re.sub("「.*?」", "", line)
            block.append(a)
            block.append(b)
        elif "『" in line and "』" in line:
            a = re.search("『(.*?)』", line).group(1)
            b = re.sub("『.*?』", "", line)
            block.append(a)
            block.append(b)
        elif "(" in line and ")" in line:
            a = re.search("\((.*?)\)", line).group(1)
            b = re.sub("\(.*?\)", "", line)
            block.append(a)
            block.append(b)
        elif "/" in line and line.count("/") == 1:
            a, b = line.split("/")
            block.append(a)
            block.append(b)
        elif "|" in line and line.count("|") == 1:
            a, b = line.split("|")
            block.append(a)
            block.append(b)
        elif re.split("\s{1,}", line).__len__() == 2:
            a, b = re.split("\s{1,}", line)
            block.append(a)
            block.append(b)
        else:
            pass
    total.append([x.strip() for x in block])

In [19]:
total = [[x for x in block if x not in names and x in titles ] for block in total ] 
total = [x for x in total if x.__len__() >= 1]
# total 

In [20]:
title_cos = {}

for block in total:
    for title in block:
        title_cos[title] = title_cos.get(title, []) + block

tmps = []
for title, cos in list(title_cos.items()):
    cos = Counter(cos)
    max_ = max(cos.values())
    tmp = pd.DataFrame({"co": list(cos.keys()), "val": list(cos.values())})
    tmp["title"] = title
    tmp = tmp[tmp.val >= 2]
    tmp["val"] /= max_
    tmps.append(tmp.sort_values(by=["val"], ascending=False)[:300])
    #print(title)
ret = pd.concat(tmps)
ret = ret[["title", "co", "val"]]
ret[ret.title == "夜は短し歩けよ乙女"]

,title,co,val
2,夜は短し歩けよ乙女,夜は短し歩けよ乙女,1.000000
175,夜は短し歩けよ乙女,舟を編む,0.102564
27,夜は短し歩けよ乙女,夜のピクニック,0.082051
37,夜は短し歩けよ乙女,図書館戦争,0.071795
107,夜は短し歩けよ乙女,人間失格,0.066667
22,夜は短し歩けよ乙女,新世界より,0.061538
66,夜は短し歩けよ乙女,蜜蜂と遠雷,0.061538
84,夜は短し歩けよ乙女,カラフル,0.061538
251,夜は短し歩けよ乙女,告白,0.056410
45,夜は短し歩けよ乙女,有頂天家族,0.056410


In [21]:
ret.to_csv("tmp/recommends_titles.csv", index=None)

In [22]:
w = ret.groupby(by=["co"]).agg(val_sum=("val", "sum")).reset_index()
df = pd.merge(ret, w, on=["co"], how="left")

df["val_norm"] = df["val"]/np.log(df["val_sum"].apply(lambda x:x)+np.e)

tmps = []
for title, s in df.groupby(by=["title"]):
    tmps.append(s.sort_values(by=["val_norm"], ascending=False))
    
df = pd.concat(tmps)
df

,title,co,val,val_sum,val_norm
46514,'','',1.000000,4.054097,0.522779
46518,'',地獄の道化師,0.250000,1.000000,0.190366
46515,'',D坂の殺人事件,0.250000,1.270202,0.180713
46516,'',二銭銅貨,0.250000,1.401351,0.176583
46522,'',時をかける少女,0.250000,1.422704,0.175941
46519,'',舞姫,0.250000,2.850847,0.145583
46523,'',七月隆文,0.250000,7.329773,0.108348
46520,'',森鴎外,0.250000,12.450522,0.091937
46517,'',何者,0.250000,15.110009,0.086782
46521,'',人間失格,0.250000,56.150048,0.061345


In [23]:
df.to_csv("tmp/recommends_titles_norm.csv", index=None)

In [24]:
df[df.title == "夜は短し歩けよ乙女"]

,title,co,val,val_sum,val_norm
17324,夜は短し歩けよ乙女,夜は短し歩けよ乙女,1.000000,32.254232,0.281329
17325,夜は短し歩けよ乙女,舟を編む,0.102564,21.267065,0.032279
17326,夜は短し歩けよ乙女,夜のピクニック,0.082051,25.192767,0.024647
17333,夜は短し歩けよ乙女,有頂天家族,0.056410,7.769658,0.024002
17350,夜は短し歩けよ乙女,鴨川ホルモー,0.035897,2.345845,0.022129
17336,夜は短し歩けよ乙女,神様のカルテ,0.051282,9.938725,0.020204
17346,夜は短し歩けよ乙女,四畳半神話大系,0.041026,5.079941,0.019975
17331,夜は短し歩けよ乙女,カラフル,0.061538,21.894598,0.019211
17335,夜は短し歩けよ乙女,夏川草介,0.051282,12.426904,0.018870
17327,夜は短し歩けよ乙女,図書館戦争,0.071795,50.582270,0.018057


In [25]:
np.log(np.e+0.1)

1.0361274185748344